In [1]:
!pip install xgbse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 349 kB 8.4 MB/s 
     |████████████████████████████████| 255.9 MB 33 kB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=cb64e1698438597f6e0b266dbd926a5fdf55117e2101ba479d8137be4003e654
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
ERROR: pip's dependency resolver does not currently take into account all t

# original

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. train_data _73.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8093115812447843
Avg. Brier Score: 0.0888623631273397


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_origin')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_origin']

# TGAN imbal 5000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7825891588030849
Avg. Brier Score: 0.09241828636888935


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-5000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-5000']

# TGAN imbal 10000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.786816693719222
Avg. Brier Score: 0.09243729766115535


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-10000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-10000']

# TGAN imbal 15000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.784927600025677
Avg. Brier Score: 0.09266216996662276


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-15000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-15000']

# TGAN imbal 20000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7881188845176851
Avg. Brier Score: 0.09223657949761137


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-20000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-20000']

# TGAN imbal 25000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7889625574293653
Avg. Brier Score: 0.09281748192520449


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-25000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-25000']

# TGAN imbal 30000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7872293598173264
Avg. Brier Score: 0.09209127342843446


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-30000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-30000']

# TGAN imbal 35000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.781947233761589
Avg. Brier Score: 0.09342126845923564


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-35000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-35000']

# TGAN imbal 40000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7860463836694269
Avg. Brier Score: 0.09340364302700307


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-40000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-40000']

# TGAN imbal 45000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.787907966289765
Avg. Brier Score: 0.09211359344553241


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-45000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-45000']

# TGAN imbal 50000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/TGAN-v7(unbalAR1&2-small)/TGAN-syn-AR-one-of-all-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7855328436362302
Avg. Brier Score: 0.09362254202496183


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-50000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-imbal-50000']

# TGAN bal 5000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7982154483846415
Avg. Brier Score: 0.09190568698500469


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-5000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-5000']

# TGAN bal 10000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7961062661054408
Avg. Brier Score: 0.0909765333935088


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-10000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-10000']

# TGAN bal 15000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7973717754729612
Avg. Brier Score: 0.0910101952193357


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-15000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-15000']

# TGAN bal 20000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7989765880767009
Avg. Brier Score: 0.0913844736626495


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-20000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-20000']

# TGAN bal 25000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7990682916540575
Avg. Brier Score: 0.09065749003524133


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-25000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-25000']

# TGAN bal 30000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8052124313369464
Avg. Brier Score: 0.08975427782128743


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-30000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-30000']

# TGAN bal 35000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8049464909626125
Avg. Brier Score: 0.08997880333689805


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-35000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-35000']

# TGAN bal 40000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8003338010215778
Avg. Brier Score: 0.09148510190695862


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-40000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-40000']

# TGAN bal 45000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7994809577521619
Avg. Brier Score: 0.09167246550882478


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-45000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-45000']

# TGAN bal 50000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7984355369702972
Avg. Brier Score: 0.09275565413422164


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-50000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-bal-50000']

# TGAN aug 5000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.803149100846424
Avg. Brier Score: 0.09407931410831716


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-5000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-5000']

# TGAN aug 10000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8063312149806964
Avg. Brier Score: 0.09315070285851738


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-1000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-1000']

# TGAN aug 15000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8089906187240364
Avg. Brier Score: 0.09323387135116183


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-15000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-15000']

# TGAN aug 20000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8061661485414546
Avg. Brier Score: 0.0943043966114063


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-20000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-20000']

# TGAN aug 25000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8094858180417618
Avg. Brier Score: 0.09448375396589345


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-25000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-25000']

# TGAN aug 30000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8077159389987804
Avg. Brier Score: 0.09391446459107416


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-30000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-30000']

# TGAN aug 35000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8114299338817207
Avg. Brier Score: 0.09296689951951194


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-35000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-35000']

# TGAN aug 40000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8113932524507781
Avg. Brier Score: 0.09240243473205287


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-40000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-40000']

# TGAN aug 45000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8102836391647639
Avg. Brier Score: 0.09345774941268534


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-45000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-45000']

# TGAN aug 50000

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8150063733986262
Avg. Brier Score: 0.0921776064943445


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-50000')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgan-aug-50000']

# TGAN org+balAR-6616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-6616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8119893257035957
Avg. Brier Score: 0.0875191893870351


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar6616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar5000']

# TGAN org+balAR-11616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-11616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8067163700055939
Avg. Brier Score: 0.08834079688882375


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar11616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar11616']

# TGAN org+balAR-16616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-16616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8059552303135346
Avg. Brier Score: 0.08838212357514147


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar16616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar16616']

# TGAN org+balAR-21616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-21616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8043320769943235
Avg. Brier Score: 0.08960798189021175


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar21616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar21616']

# TGAN org+balAR-26616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-26616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8041394994818748
Avg. Brier Score: 0.08951007461749101


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar26616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar26616']

# TGAN org+balAR-31616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-31616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8074775096976533
Avg. Brier Score: 0.08915418349849953


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar31616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar31616']

# TGAN org+balAR-36616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-36616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8092932405293131
Avg. Brier Score: 0.08859696814535874


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar36616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar36616']

# TGAN org+balAR-41616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-41616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8063862371271103
Avg. Brier Score: 0.0900363256060958


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar41616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar41616']

# TGAN org+balAR-46616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-46616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8026539015286986
Avg. Brier Score: 0.09092981277056492


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar46616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar46616']

# TGAN org+balAR-51616

In [ ]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-51616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [ ]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [ ]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [ ]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [ ]:
y_pred = xgbse_model.predict(X_test)

In [ ]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.7962805029024183
Avg. Brier Score: 0.09221372601372935


In [ ]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [ ]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar51616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_org+tgan-balar51616']

# TGAN aug + origin-6616

In [2]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-6616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [4]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [6]:
data_train = new_train_df
data_test = new_test_df

In [7]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [8]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [9]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [10]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [11]:
y_pred = xgbse_model.predict(X_test)

In [12]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8125853989564132
Avg. Brier Score: 0.08980875825790915


In [13]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [14]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-6616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-6616']

# TGAN aug + origin-11616

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-11616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8144103001458087
Avg. Brier Score: 0.08964474021048625


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-11616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-11616']

# TGAN aug + origin-16166

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-16166.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.814749603382028
Avg. Brier Score: 0.08936569143242859


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-16616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-16616']

# TGAN aug + origin-21616

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-21616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.814667070162407
Avg. Brier Score: 0.09053809111182029


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-21616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-21616']

# TGAN aug + origin-26616

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-26616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.816528652782745
Avg. Brier Score: 0.09056248475509483


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-26616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-26616']

# TGAN aug + origin-31616

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-31616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.815519913431823
Avg. Brier Score: 0.09046688685843604


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-31616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-31616']

# TGAN aug + origin-36616

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-36616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.818500279695911
Avg. Brier Score: 0.09003176485225642


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-36616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-36616']

# TGAN aug + origin-41616

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-41616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8145937073005218
Avg. Brier Score: 0.09076011514008807


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-41616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-41616']

# TGAN aug + origin-46616

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-46616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8134840940145075
Avg. Brier Score: 0.09171087860050205


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-46616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-46616']

# TGAN aug + origin-51616

In [1]:
from xgbse.converters import convert_to_structured
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-51616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
X_train = data_train.drop(['a_year_time', 'a_year_event'], axis=1)
T_train = data_train['a_year_time']
E_train = data_train['a_year_event']
y_train = convert_to_structured(T_train, E_train)

In [7]:
X_test = data_test.drop(['a_year_time', 'a_year_event'], axis=1)
T_test = data_test['a_year_time']
E_test = data_test['a_year_event']
y_test = convert_to_structured(T_test, E_test)

In [8]:
TIME_BINS = np.arange(15, 315, 15)
TIME_BINS

array([ 15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180, 195,
       210, 225, 240, 255, 270, 285, 300])

In [9]:
from xgbse import XGBSEDebiasedBCE

xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X_train, y_train, time_bins=TIME_BINS)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [10]:
y_pred = xgbse_model.predict(X_test)

In [11]:
from xgbse.metrics import concordance_index, approx_brier_score

# running metrics
print(f"C-index: {concordance_index(y_test, y_pred)}")
print(f"Avg. Brier Score: {approx_brier_score(y_test, y_pred)}")

C-index: 0.8151897805533393
Avg. Brier Score: 0.09113683183927776


In [12]:
from joblib import dump, load
from xgbse import XGBSEStackedWeibull

In [13]:
model = XGBSEStackedWeibull()

dump(model, '/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-46616')

['/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/xgbse/xgbse_new(onehot)_tgna-aug-orig-46616']